In [1]:
import numpy as np
from scipy.spatial import ConvexHull

v1 = np.array([[0.0, 0.0,0.0,0.0],[1.0, 1.0,1.0,1.0], [1.0/2, 1.0/2,1.0/2,1.0/2], [1.0/4, 1.0/4,1.0/4,1.0/4]])
v2 = np.array([[0.0, 0.0,0.0,0.0],[1.0, 1.0,1.0,1.0]]) # good
v3 = np.array([[1.0, 1.0,0.0,1.0],[1.0, 0.0,2.0,1.0]])
v4 = np.array([[1.0,0.0,1.0],[ 0.0,2.0,1.0], [ 0.0,0.0,1.0], [ 1.0,2.0,1.0]])
v5 = np.zeros((2,4))
vn = np.array([[0.0,0.0,0.0],[1.0,1.0,0.0],[0.0,0.0,2.0],[1.0,1.0,2.0]])
vline = np.array([[1.0,1.0,1.0],[2.0,2.0,2.0]])

v = vline
m_ = v[0]
v = v - m_
U,S,V = np.linalg.svd(v)
epsilon = 0.000001
npoints, ndim = v.shape


In [2]:
print (v)
print (U,S,V)

[[0. 0. 0.]
 [1. 1. 1.]]
[[ 0. -1.]
 [-1.  0.]] [1.73205081 0.        ] [[-0.57735027 -0.57735027 -0.57735027]
 [ 0.81649658 -0.40824829 -0.40824829]
 [ 0.         -0.70710678  0.70710678]]


In [3]:
print (S)
if not (np.abs(S) < epsilon).any():
    dim_to_remove = S.shape[0]
else:
    dim_to_remove = np.argmax(np.abs(S) < epsilon)
print (dim_to_remove)
diag_l = S.shape[0]
reconstruct_diag = np.zeros((npoints, ndim))
reconstruct_diag[:diag_l,:diag_l] = np.diag(S)

new_pts = np.matmul(U,reconstruct_diag)
print (new_pts)
print (new_pts[:, :dim_to_remove])

[1.73205081 0.        ]
1
[[ 0.          0.          0.        ]
 [-1.73205081  0.          0.        ]]
[[ 0.        ]
 [-1.73205081]]


In [4]:
def toHull(low_dim_pts_reduct):
    if low_dim_pts_reduct.shape[1] == 1:
        lb = np.min(low_dim_pts_reduct)
        ub = np.max(low_dim_pts_reduct)
        # x <= ub  -x >= -lb
        A = np.array([[1.0],[-1.0]])
        b = np.array([ub, -lb])
    else:
        # convex hull will get you Ax + b <= 0
        hull = ConvexHull(points=low_dim_pts_reduct)
        A = hull.equations[:,:-1]
        b = -hull.equations[:,-1]
    print (A,b)
    return A,b

In [5]:
low_dim_pts_reduct = new_pts[:, :dim_to_remove]
if dim_to_remove != 0:
    A,b = toHull(low_dim_pts_reduct)
    # complete to the full dimension
    n_rows = A.shape[0]
    n_cols = A.shape[1]

    num_var_to_add = ndim - (dim_to_remove)

    zero4 = np.zeros((n_rows,num_var_to_add))
    rows_to_append = np.zeros((2*num_var_to_add, n_cols+num_var_to_add))
    for idx in range(num_var_to_add):
        rows_to_append[idx*2,n_cols+ idx] = 1
        rows_to_append[idx*2+1,n_cols+ idx] = -1

    trueA = np.vstack([np.hstack([A, zero4]), rows_to_append])
    trueB = np.append(b, [0.0, 0.0]*num_var_to_add)
else:
    #num_var_to_add = ndim # v.shape[1] - (dim_to_remove)  # v.shape[1] 
    # directly construct trueA trueB
    trueA = np.zeros((ndim*2, ndim))
    for idx in range(ndim):
        trueA[idx*2,idx] = 1
        trueA[idx*2+1,idx] = -1
    trueB = np.zeros(ndim*2)

    
print (trueA)
print (trueB)

[[ 1.]
 [-1.]] [0.         1.73205081]
[[ 1.  0.  0.]
 [-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]
 [ 0.  0. -1.]]
[0.         1.73205081 0.         0.         0.         0.        ]


In [6]:
finalA = np.matmul(trueA, V)
finalA

array([[-0.57735027, -0.57735027, -0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.81649658, -0.40824829, -0.40824829],
       [-0.81649658,  0.40824829,  0.40824829],
       [ 0.        , -0.70710678,  0.70710678],
       [ 0.        ,  0.70710678, -0.70710678]])

In [7]:
finalB = np.matmul(finalA, m_) + trueB

In [8]:
finalB

array([-1.73205081e+00,  3.46410162e+00,  2.22044605e-16, -2.22044605e-16,
        0.00000000e+00,  0.00000000e+00])

In [ ]:
-0.78867513/0.21132487

In [ ]:
a_ = 0.57735027; b_ = 0.78867513; c_ = 0.21132487;
(b_ - c_)/(a_)

In [ ]:
a_+c_-b_